# Understanding Housing in California

Group members: Ruben, Jiyang, Kaia
Date December 2, 2025
Data Visualization | Prof Rodden, GSI Yazen | MaCSS 2025


**Research questions:**
- Landscape: How is affordability distributed?
- The “Gravity of Affordability” :
         H1: Low affordability pushes people out 
         H2:  High demand impacts affordability 
- Does any market adjustment mechanism on construction supply respond to price signals?



**Background**
- We are using the secondary dataset on construction permit numbers to reveal the supply-demand relationship. -> Housing and Urban Development Dataset
- We limit our scope to the state of California so we can dive deeper into the county level. -> Redfin, HUD, and NIH datasets.
- If possible, we intend to combine demographic data into our analysis to see how demographic patterns interact with the housing market. -> NIH population and income datasest

## Data Preparation
**Datasets:** 

1. HUD (Housing and Urban Development) Housing Construction Permits [https://hudgis-hud.opendata.arcgis.com/datasets/HUD::residential-construction-permits-by-county/aboutLinks]
- New residential building permits by county, 1980-2022
- 3221 rows but more than 200 columns (wide format, 6 variables per year)
- Annual counts of building permits for new housing construction, broken down by single-family vs. multi-family units, with county-level geographic detail


2. Redfin Monthly Housing Market Data [https://www.redfin.com/news/data-center]
- Housing market activity across US counties, including home prices, inventory levels, sales counts, days on market, and other market indicators. 
- Coverage: United States, county-level, 2012-present (monthly data)


3. HDPulse (2019-2023 )
An Ecosystem of Health Disparities and Minority Health Resources. We are using median household income data and population data by county from this dataset.
[https://hdpulse.nimhd.nih.gov/data-portal/social/map?age=999&age_options=ageall_1&demo=00102&demo_options=pop_12&race=00&race_options=raceall_1&sex=0&sex_options=sexboth_1&socialtopic=070&socialtopic_options=social_6&statefips=06&statefips_options=area_states] 



- Understanding the variables: <br>
Most metrics have the following variables: the base value, "MOM" (month-over-month % change), "POP" (population size), INCOME (income by county)  and "YOY" (year-over-year % change)<br>

In [ ]:
install.packages(c("ggplot2", "dplyr", "maps", "RColorBrewer", "ggrepel", "stringr", "tidyr"))

In [ ]:
library(readr)
library(dplyr)
library(ggplot2)
library(scales)
library(RColorBrewer)
library(lubridate)
library(ggrepel)
library(stringr)

In [ ]:
redfin <- read_tsv("/hex/county_market_tracker.tsv000 (1)") # Redfin monthly housing data
hud <- read_csv("/hex/Residential_Construction_Permits_by_County_5026727375813176131.csv") # US department of housing and urban developmeny Permits issued data
income <- read_csv("/hex/CA income for counties.csv") # Income across counties in California  
population <- read_csv("/hex/Population_by_counties.csv") # Population size across California counties

For each dataset we created distinact dataframes

We used specific names to refenrece each dataframe

hud = housing and urban development<br>
redfin = redfin data<br>
pop and income = california specific annual household income and popualtion by county

In [ ]:
redfin %>%
  filter(STATE == "California", PROPERTY_TYPE != "All Residential") %>%
  mutate(
    year = year(PERIOD_END),
    type_group = if_else(PROPERTY_TYPE == "Single Family Residential",
                         "Single Family Residential", "Other")
  )

In [ ]:
# Clean & prepare keys
redfin_clean <- redfin %>%
  select(REGION, STATE, MEDIAN_SALE_PRICE) %>%
  mutate(
    REGION = tolower(REGION),
    STATE = tolower(STATE)
  )

hud_clean <- hud %>%
  select(COUNTY, STATE_NAME, GEOID, ALL_PERMITS_1980) %>%
  mutate(
    COUNTY = tolower(COUNTY),
    STATE_NAME = tolower(STATE_NAME)
  )

# Merge by county + state 
merged <- redfin_clean %>%
  left_join(hud_clean, by = c("REGION" = "COUNTY", "STATE" = "STATE_NAME"))

# Quick check 
head(merged)

In [ ]:
# remove the two most expensive homes
merged <- merged %>%
  filter(MEDIAN_SALE_PRICE < 500000000)
# check the data again
merged %>%
  arrange(desc(MEDIAN_SALE_PRICE)) %>%
  slice_head(n = 20)

In [ ]:
hud_df <- hud %>%
    mutate(
        total_permits = rowSums(across(matches("^ALL_PERMITS_\\d{4}$")), na.rm = TRUE)
    )

In [ ]:
hud_df_ca <- hud_df %>%
    filter(STUSAB == "CA")

In [ ]:
hud_long <- hud %>%
  filter(STUSAB == "CA") %>%
  select(STUSAB, STATE_NAME, COUNTY, starts_with("SINGLE_FAMILY"), 
         starts_with("ALL_MULTIFAMILY")) %>%
  pivot_longer(
    cols = -c(STUSAB, STATE_NAME, COUNTY),
    names_to = c("permit_type", "year"),
    names_pattern = "(.*)_PERMITS_(\\d+)",
    values_to = "permits"
  ) %>%
  mutate(
    year = as.integer(year),
    permit_type = case_when(
      permit_type == "SINGLE_FAMILY" ~ "Single-family",
      permit_type == "ALL_MULTIFAMILY" ~ "Multifamily",
      TRUE ~ permit_type
    )
  )

In [ ]:
income_clean <- income %>%
  slice(-1, -2) %>%
  rename(
    county = County,
    fips = FIPS,
    median_income = `Value (Dollars)`
  )

population_clean <- population %>%
  slice(-1, -2) %>%
  rename(
    county = County,
    population = `People(all age)`
  )

### Home Sales Price compared to Price of Homes (Price reslience of homes versus Volume of homes)

In [ ]:
plot_data <- redfin %>%
  filter(STATE == "California", PROPERTY_TYPE == "All Residential") %>%
  mutate(year = year(PERIOD_BEGIN)) %>%
  group_by(year) %>%
  summarise(
    total_sales = sum(HOMES_SOLD, na.rm = TRUE),
    median_price = median(MEDIAN_SALE_PRICE, na.rm = TRUE),
    .groups = "drop"
  )

scale_factor <- max(plot_data$total_sales, na.rm = TRUE) / max(plot_data$median_price, na.rm = TRUE)

plot_data %>%
  ggplot(aes(x = factor(year))) +
  geom_col(aes(y = total_sales), fill = "#332b45", width = 0.9) +
  geom_line(aes(y = median_price * scale_factor),
            color = "#bd6877", size = 1.2, group = 1) +
  scale_y_continuous(
    name = "Total Homes Sold(Bar)",
    labels = scales::comma,
    sec.axis = sec_axis(~ . / scale_factor,
                        name = "Median Sale Price (Line)", labels = scales::dollar)
  ) +
  labs(
    title = "Volume Volatility vs. Price Resilience",
    x = "Year",
    y = "Total Homes Sold"
  ) +
  theme_minimal() +
  theme(
    plot.title = element_text(hjust = 0.5, size = 13),
    legend.position = "none",
    axis.text.x = element_text(angle = 45, hjust = 1)
  ) +
  theme(        
    plot.background = element_rect(fill = "white", color = NA), # Set background color
    panel.spacing = unit(1.2, "lines"),         # Adjust space between panels if faceted
    aspect.ratio = 0.6                          # Control plot aspect ratio (height/width)
  )

### Construction Permit Analysis

In [ ]:
sales_data <- redfin %>%
  filter(STATE == "California") %>%
  mutate(year = year(PERIOD_END)) %>%
  group_by(year) %>%
  summarise(value = sum(HOMES_SOLD, na.rm = TRUE), .groups = "drop")

permits_data <- hud_long %>%
  group_by(year) %>%
  summarise(value = sum(permits, na.rm = TRUE), .groups = "drop")

### San Francisco County Price-to-Income Ratio

In [ ]:
sales_data <- redfin %>%
  filter(
    STATE == "California",
    REGION_TYPE == "county",
    REGION == "San Francisco County, CA"
  ) %>%
  mutate(year = year(PERIOD_END)) %>%
  group_by(year) %>%
  summarise(value = sum(HOMES_SOLD, na.rm = TRUE), .groups = "drop")

permits_data <- hud_df %>%
  filter(STUSAB == "CA", NAME == "San Francisco") %>%
  select(starts_with("ALL_PERMITS_")) %>%
  pivot_longer(
    cols = everything(),
    names_to = "perm_year",
    values_to = "permits"
  ) %>%
  mutate(year = as.integer(sub("^ALL_PERMITS_", "", perm_year))) %>%
  group_by(year) %>%
  summarise(value = sum(permits, na.rm = TRUE), .groups = "drop")

common_years <- intersect(sales_data$year, permits_data$year)

sales_data <- sales_data %>% filter(year %in% common_years)
permits_data <- permits_data %>% filter(year %in% common_years)

scale_factor <- max(sales_data$value) / max(permits_data$value) * 0.6

ggplot() +
  geom_line(data = sales_data, aes(x = year, y = value, color = "Homes Sold"), linewidth = 1.2) +
  geom_point(data = sales_data, aes(x = year, y = value, color = "Homes Sold"), size = 2.5) +
  geom_line(data = permits_data, aes(x = year, y = value * scale_factor, color = "Building Permits"), linewidth = 1.2) +
  geom_point(data = permits_data, aes(x = year, y = value * scale_factor, color = "Building Permits"), size = 2.5) +
  labs(
    title = "San Francisco County",
    x = "Year",
    y = "Homes Sold",
    color = ""
  ) +
  scale_x_continuous(breaks = seq(min(common_years), max(common_years), by = 1)) +
  scale_y_continuous(
    labels = comma,
    sec.axis = sec_axis(~ . / scale_factor, name = "Building Permits", labels = comma)
  ) +
  theme_minimal() +
  theme(
    plot.title = element_text(hjust = 0.5, size = 13, face = "bold"),
    legend.position = "top",
    panel.grid.major.x = element_line(color = "gray85", linewidth = 0.5),
    panel.grid.minor.x = element_blank()
  )

### Los Angeles County Price-to-Income Ratio

In [ ]:
sales_data <- redfin %>%
  filter(
    STATE == "California",
    REGION_TYPE == "county",
    REGION == "Los Angeles County, CA"
  ) %>%
  mutate(year = year(PERIOD_END)) %>%
  group_by(year) %>%
  summarise(value = sum(HOMES_SOLD, na.rm = TRUE), .groups = "drop")

permits_data <- hud_df %>%
  filter(STUSAB == "CA", NAME == "Los Angeles") %>%
  select(starts_with("ALL_PERMITS_")) %>%
  pivot_longer(
    cols = everything(),
    names_to = "perm_year",
    values_to = "permits"
  ) %>%
  mutate(year = as.integer(sub("^ALL_PERMITS_", "", perm_year))) %>%
  group_by(year) %>%
  summarise(value = sum(permits, na.rm = TRUE), .groups = "drop")

common_years <- intersect(sales_data$year, permits_data$year)

sales_data <- sales_data %>% filter(year %in% common_years)
permits_data <- permits_data %>% filter(year %in% common_years)

scale_factor <- max(sales_data$value) / max(permits_data$value) * 0.6

ggplot() +
  geom_line(data = sales_data, aes(x = year, y = value, color = "Homes Sold"), linewidth = 1.2) +
  geom_point(data = sales_data, aes(x = year, y = value, color = "Homes Sold"), size = 2.5) +
  geom_line(data = permits_data, aes(x = year, y = value * scale_factor, color = "Building Permits"), linewidth = 1.2) +
  geom_point(data = permits_data, aes(x = year, y = value * scale_factor, color = "Building Permits"), size = 2.5) +
  labs(
    title = "Los Angeles County",
    x = "Year",
    y = "Homes Sold",
    color = ""
  ) +
  scale_x_continuous(breaks = seq(min(common_years), max(common_years), by = 1)) +
  scale_y_continuous(
    labels = comma,
    sec.axis = sec_axis(~ . / scale_factor, name = "Building Permits", labels = comma)
  ) +
  theme_minimal() +
  theme(
    plot.title = element_text(hjust = 0.5, size = 13, face = "bold"),
    legend.position = "top",
    panel.grid.major.x = element_line(color = "gray85", linewidth = 0.5),
    panel.grid.minor.x = element_blank()
  )

### Riverside County Price-to-Income Ratio

In [ ]:
sales_data <- redfin %>%
  filter(
    STATE == "California",
    REGION_TYPE == "county",
    REGION == "Riverside County, CA"
  ) %>%
  mutate(year = year(PERIOD_END)) %>%
  group_by(year) %>%
  summarise(value = sum(HOMES_SOLD, na.rm = TRUE), .groups = "drop")

permits_data <- hud_df %>%
  filter(STUSAB == "CA", NAME == "Riverside") %>%
  select(starts_with("ALL_PERMITS_")) %>%
  pivot_longer(
    cols = everything(),
    names_to = "perm_year",
    values_to = "permits"
  ) %>%
  mutate(year = as.integer(sub("^ALL_PERMITS_", "", perm_year))) %>%
  group_by(year) %>%
  summarise(value = sum(permits, na.rm = TRUE), .groups = "drop")

common_years <- intersect(sales_data$year, permits_data$year)

sales_data <- sales_data %>% filter(year %in% common_years)
permits_data <- permits_data %>% filter(year %in% common_years)

scale_factor <- max(sales_data$value) / max(permits_data$value) * 0.6

ggplot() +
  geom_line(data = sales_data, aes(x = year, y = value, color = "Homes Sold"), linewidth = 1.2) +
  geom_point(data = sales_data, aes(x = year, y = value, color = "Homes Sold"), size = 2.5) +
  geom_line(data = permits_data, aes(x = year, y = value * scale_factor, color = "Building Permits"), linewidth = 1.2) +
  geom_point(data = permits_data, aes(x = year, y = value * scale_factor, color = "Building Permits"), size = 2.5) +
  labs(
    title = "Riverside County",
    x = "Year",
    y = "Homes Sold",
    color = ""
  ) +
  scale_x_continuous(breaks = seq(min(common_years), max(common_years), by = 1)) +
  scale_y_continuous(
    labels = comma,
    sec.axis = sec_axis(~ . / scale_factor, name = "Building Permits", labels = comma)
  ) +
  theme_minimal() +
  theme(
    plot.title = element_text(hjust = 0.5, size = 13, face = "bold"),
    legend.position = "top",
    panel.grid.major.x = element_line(color = "gray85", linewidth = 0.5),
    panel.grid.minor.x = element_blank()
  )

### Lassen County Price-to-Income Ratio

In [ ]:
sales_data <- redfin %>%
  filter(
    STATE == "California",
    REGION_TYPE == "county",
    REGION == "Lassen County, CA"
  ) %>%
  mutate(year = year(PERIOD_END)) %>%
  group_by(year) %>%
  summarise(value = sum(HOMES_SOLD, na.rm = TRUE), .groups = "drop")

permits_data <- hud_df %>%
  filter(STUSAB == "CA", NAME == "Lassen") %>%
  select(starts_with("ALL_PERMITS_")) %>%
  pivot_longer(
    cols = everything(),
    names_to = "perm_year",
    values_to = "permits"
  ) %>%
  mutate(year = as.integer(sub("^ALL_PERMITS_", "", perm_year))) %>%
  group_by(year) %>%
  summarise(value = sum(permits, na.rm = TRUE), .groups = "drop")

common_years <- intersect(sales_data$year, permits_data$year)

sales_data <- sales_data %>% filter(year %in% common_years)
permits_data <- permits_data %>% filter(year %in% common_years)

scale_factor <- max(sales_data$value) / max(permits_data$value) * 0.6

ggplot() +
  geom_line(data = sales_data, aes(x = year, y = value, color = "Homes Sold"), linewidth = 1.2) +
  geom_point(data = sales_data, aes(x = year, y = value, color = "Homes Sold"), size = 2.5) +
  geom_line(data = permits_data, aes(x = year, y = value * scale_factor, color = "Building Permits"), linewidth = 1.2) +
  geom_point(data = permits_data, aes(x = year, y = value * scale_factor, color = "Building Permits"), size = 2.5) +
  labs(
    title = "Lassen County",
    x = "Year",
    y = "Homes Sold",
    color = ""
  ) +
  scale_x_continuous(breaks = seq(min(common_years), max(common_years), by = 1)) +
  scale_y_continuous(
    labels = comma,
    sec.axis = sec_axis(~ . / scale_factor, name = "Building Permits", labels = comma)
  ) +
  theme_minimal() +
  theme(
    plot.title = element_text(hjust = 0.5, size = 13, face = "bold"),
    legend.position = "top",
    panel.grid.major.x = element_line(color = "gray85", linewidth = 0.5),
    panel.grid.minor.x = element_blank()
  )

In [ ]:
# In SF, LA, and Riverside counties, it would take many years of total income to buy the median priced home in each county.
# This chart shows that homes are out of reach for people in a given county given the median home sale price and median HHI
# Compute price-to-income ratio (PIR): a standard affordability metric

pir_df <- pop_income_merged %>%
  mutate(
    pir = median_sale_price_2019_2023 / median_income
  )

# Filter for SF, LA, and Riverside counties
target_counties <- c("San Francisco County", "Los Angeles County", "Riverside County")

# Find counties with max and min PIR
max_pir_county <- pir_df %>%
  filter(pir == max(pir, na.rm = TRUE)) %>%
  pull(county)

min_pir_county <- pir_df %>%
  filter(pir == min(pir, na.rm = TRUE)) %>%
  pull(county)

# Combine target counties with max/min PIR counties
all_target_counties <- unique(c(target_counties, max_pir_county, min_pir_county))

target_data <- pir_df %>%
  filter(county %in% all_target_counties)

# Plot home price vs. 3× income (affordability threshold)
pir_df %>%
  ggplot(aes(x = median_income, y = median_sale_price_2019_2023, size = population)) +
  geom_point(alpha = 0.2, color = "gray60") +
  geom_point(
    data = target_data,
    aes(x = median_income, y = median_sale_price_2019_2023, color = county),
  ) +
  geom_text_repel(
    data = target_data,
    aes(label = paste0(county, "\nPIR = ", round(pir, 1)), color = county),
    fontface = "bold",
    size = 4,
    nudge_y = 80000,
    show.legend = FALSE
  ) +
  scale_color_manual(
    values = c(
      "San Francisco County" = "#b34b4b",
      "Los Angeles County" = "#c54b49",
      "Riverside County" = "#e47969",
      "Lassen County" = "#f4b790"
    ),
    breaks = all_target_counties
  ) +
  geom_abline(
    slope = 3,
    intercept = 0,
    linetype = "dashed",
    color = "black",
    linewidth = 1,
    alpha = 0.8
  ) +
  annotate(
    "text",
    x = min(pir_df$median_income) * 1.25,
    y = min(pir_df$median_income) * 3.2,
    label = "Affordability Threshold (3X Income)",
    hjust = 0,
    fontface = "italic",
    size = 3.5
  ) +
  scale_x_continuous(labels = scales::dollar) +
  scale_y_continuous(labels = scales::dollar) +
  labs(
    title = "Home prices far exceed what income can support",
    subtitle = "Using price-to-income ratio (PIR) to calculate gap",
    x = "Median Household Income",
    y = "Median Sale Price",
    color = NULL
  ) +
  theme_minimal() +
  theme(
    legend.position = "none",
    aspect.ratio = 0.6
  )

## Choropleth
GIF uploaded to files
![Choropleth Map](assets/choropleth.gif)